In [3]:
from flowstab.flow_stability import FlowStability
import pandas as pd
import numpy as np
from tqdm import tqdm
from flowstab.network_clustering import avg_norm_var_information

df=pd.read_csv('Pre_processed_Data\primary_school_day2.csv')
fs = FlowStability()
fs.set_temporal_network(events_table=df, relabel_nodes=True)
fs.compute_laplacian_matrices()
taus=np.logspace(-2, 6, 4)
fs.set_time_scale(taus)
fs.compute_inter_transition_matrices()
fs.time_direction = 0  # perform both forward and backard
fs.set_flow_clustering()

2025-11-11 17:58:49,013 - INFO - flowstab\flow_stability.py:235 - PID:23404 - Set an empty temporal network.
2025-11-11 17:58:49,013 - INFO - flowstab\flow_stability.py:139 - PID:23404 - Successfully initiated a FlowStability instance!
2025-11-11 17:58:49,024 - INFO - flowstab\flow_stability.py:230 - PID:23404 - Set the temporal network: <class 'flowstab.temporal_network.ContTempNetwork'> with 238 nodes and 65150 events
2025-11-11 17:58:49,024 - INFO - flowstab\temporal_network.py:1176 - PID:23404 - Computing Laplacians
2025-11-11 17:58:49,075 - INFO - flowstab\temporal_network.py:1266 - PID:23404 - 0 over 1545
2025-11-11 17:58:49,075 - INFO - flowstab\temporal_network.py:1270 - PID:23404 - 0.00s
2025-11-11 17:59:10,414 - INFO - flowstab\temporal_network.py:1266 - PID:23404 - 1000 over 1545
2025-11-11 17:59:10,415 - INFO - flowstab\temporal_network.py:1270 - PID:23404 - 21.34s
2025-11-11 17:59:19,398 - INFO - flowstab\temporal_network.py:1326 - PID:23404 - Finished in 30.32302069664001

<class 'flowstab.flow_stability.FlowStability'>
- temporal network: <class 'flowstab.temporal_network.ContTempNetwork'> with 238 nodes and 65150 events
- t_start: None
- t_stop: None
- time_scale: <iterator object at 0x000001CAD73FADA0>

In [ ]:
forward_partitions={i: {'clusters':[], 'stability_vals':[]} for i in taus}
backward_partitions={i: {'clusters':[], 'stability_vals':[]} for i in taus}
for tau in taus):
    for i in tqdm(range(200)):
        #Forward
        fs.flow_clustering_forward[tau].clustering={}
        fs.flow_clustering_forward[tau].find_louvain_clustering()
        clustering_for=fs.flow_clustering_forward[tau].clustering[0]
        forward_partitions[tau]['clusters'].append(clustering_for.partition.cluster_list)
        stab_value_for=clustering_for.compute_stability()
        forward_partitions[tau]['stability_vals'].append(stab_value_for)


        #Backward
        fs.flow_clustering_backward[tau].clustering={}
        fs.flow_clustering_backward[tau].find_louvain_clustering()
        clustering_back=fs.flow_clustering_backward[tau].clustering[0]
        backward_partitions[tau]['clusters'].append(clustering_back.partition.cluster_list)
        stab_value_back=clustering_back.compute_stability()
        backward_partitions[tau]['stability_vals'].append(stab_value_back)

  0%|          | 0/4 [00:20<?, ?it/s]


KeyboardInterrupt: 

In [14]:
for tau in taus:
    print(tau, avg_norm_var_information(forward_partitions[tau]['clusters']))

0.01 0.0018264485280535014
4.6415888336127775 0.0059890413611124295
2154.4346900318824 0.014594210130516322
1000000.0 0.0002692968240332838


In [ ]:

        clustering_back=find_louvain(clustering_object_back)
        backward_partitions[tau]['clusters'].append(clustering_back)
        stab_value_back=clustering_back.compute_stability()
        backward_partitions[tau]['stability_vals'].append(stab_value_back)